In [32]:
# %reset
from sympy import *
import numpy as np 
init_printing()

comp, out_of_plane = symbols( 'comp, out_of_plane' )

iso = 1

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44 = symbols('D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44')
Exx, Ezz, Gxz, divV = symbols('Exx, Ezz, Gxz, divV')
stress_labels = ['Txx', 'Tzz', 'Txz', 'P']

if iso==1:
    # Isotropic
    Dv = Matrix( [ [D11, 0, 0, 0], [0, D22, 0, 0], [0, 0, D33, 0], [0, 0, 0, 0]  ] )
else:
    # Anisotropic
    Dv = Matrix( [ [D11, D12, D13, D14], [D21, D22, D23, D24], [D31, D32, D33, D34], [D41, D42, D43, D44]  ] )

# Check constitutive
Ed = Matrix( [ [Exx], [Ezz], [Gxz], [divV] ] )
T  = Dv*Ed
for i in range(4):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )

Txx = D11*Exx;
Tzz = D22*Ezz;
Txz = D33*Gxz;
P = 0;


In [48]:
# Compressibility switch
comp = symbols('comp')
comp  = 1

SxxBC = symbols('SxxBC') # boundary stress

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')
dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension 1
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')
dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pW, pE, pSW, pSE, pNW, pNE = symbols('pW, pE, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pW, pE, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
NeuN, NeuS, DirN, DirS  = symbols('NeuN, NeuS, DirN, DirS')

dVxdzN_in  = (uN - uC )/dz
dVxdzS_in  = (uC - uS )/dz
dVxdzN_Dir = ((-uC) - uC )/dz
dVxdzS_Dir = (uC - (-uC) )/dz
dVxdzN_Neu = ((uC) - uC )/dz
dVxdzS_Neu = (uC - (uC) )/dz

# Divergences
divW      = inW  *out_of_plane*( (uC  - uW )/dx + (vNW - vSW )/dz ) 
divE      = inE  *out_of_plane*( (uE  - uC )/dx + (vNE - vSE )/dz )
divSW     = inSWc*out_of_plane*( (uS  - uSW)/dx + (vSW - vSSW)/dz )
divSE     = inSEc*out_of_plane*( (uSE - uS )/dx + (vSE - vSSE)/dz )
divNW     = inNWc*out_of_plane*( (uN  - uNW)/dx + (vNNW- vNW )/dz )
divNE     = inNEc*out_of_plane*( (uNE - uN )/dx + (vNNE- vNE )/dz )

# Deviatoric normal strain rate
ExxW      = inW  *((uC -uW )/dx - comp*1/3*divW) 
EzzW      = inW  *((vNW-vSW)/dz - comp*1/3*divW)
ExxE      = inE  *((uE -uC )/dx - comp*1/3*divE)
EzzE      = inE  *((vNE-vSE)/dz - comp*1/3*divE)

# Additional missing stress tensor components needed for interpolation
GxzNE     = inNEv*((uNE - uE )/dz + ( vNEE - vNE  )/dx)
GxzSE     = inSEv*((uE  - uSE)/dz + ( vSEE - vSE  )/dx)
GxzNW     = inNWv*((uNW - uW )/dz + ( vNW  - vNWW )/dx)
GxzSW     = inSWv*((uW  - uSW)/dz + ( vSW  - vSWW )/dx)

# Shear strain rate
GxzN      = ( inN*dVxdzN_in + DirS*dVxdzN_Dir + NeuS*dVxdzN_Neu + (vNE -vNW )/dx)        
GxzS      = ( inS*dVxdzS_in + DirN*dVxdzS_Dir + NeuN*dVxdzS_Neu + (vSE -vSW )/dx) 
ExxNE     = inNEc*((uNE -uN )/dx - comp*1/3*divNE)
ExxNW     = inNWc*((uN  -uNW)/dx - comp*1/3*divNW)
ExxSE     = inSEc*((uSE -uS )/dx - comp*1/3*divSE)
ExxSW     = inSWc*((uS  -uSW)/dx - comp*1/3*divSW)
EzzNE     = inNEc*((vNNE-vNE)/dz - comp*1/3*divNE)
EzzNW     = inNWc*((vNNW-vNW)/dz - comp*1/3*divNW)
EzzSE     = inSEc*((vSE-vSSE)/dz - comp*1/3*divSE)
EzzSW     = inSWc*((vSW-vSSW)/dz - comp*1/3*divSW)
# INV 0
GxzE      = 0.25*( inN*GxzN + inS*GxzS + inNEv*GxzNE + inSEv*GxzSE)      # clear
GxzW      = 0.25*( inN*GxzN + inS*GxzS + inNWv*GxzNW + inSWv*GxzSW)
ExxN      = 0.25*( inE*ExxE + inW*ExxW + inNWc*ExxNW + inNEc*ExxNE)
ExxS      = 0.25*( inE*ExxE + inW*ExxW + inSWc*ExxSW + inSEc*ExxSE)
EzzN      = 0.25*( inE*EzzE + inW*EzzW + inNWc*EzzNW + inNEc*EzzNE) 
EzzS      = 0.25*( inE*EzzE + inW*EzzW + inSWc*EzzSW + inSEc*EzzSE)
pS        = 0.25*( inE*pE   + inW*pW   + inSWc*pSW   + inSEc*pSE)
pN        = 0.25*( inE*pE   + inW*pW   + inSWc*pNW   + inSEc*pNE)  

#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W', D14:'D14W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E', D14:'D14E'})
#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S', D34:'D34S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N', D34:'D34N'})
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS) - 1/dx*(pE - pW)
Fx  = -Fx

# Stress boundary
SxxE = TxxE - pE
SxxW = 2*SxxBC - SxxE
Fx  = 1/dx*(SxxE - SxxW) + 1/dz*(TxzN - TxzS)*2 # dodgy factor 2 make it symmetric
Fx  = -Fx/2


for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cUc)) + ';')

uW = D11W*inW*(0.33333333333333331*inW*out_of_plane - 1)/pow(dx, 2);
uC = (pow(dx, 2)*(D33N*(2*DirS + inN) + D33S*(2*DirN + inS)) - pow(dz, 2)*(D11E*inE*(0.33333333333333331*inE*out_of_plane - 1) + D11W*inW*(0.33333333333333331*inW*out_of_plane - 1)))/(pow(dx, 2)*pow(dz, 2));
uE = D11E*inE*(0.33333333333333331*inE*out_of_plane - 1)/pow(dx, 2);
uS = -D33S*inS/pow(dz, 2);
uN = -D33N*inN/pow(dz, 2);
vSW = (0.33333333333333331*D11W*pow(inW, 2)*out_of_plane - D33S)/(dx*dz);
vSE = (-0.33333333333333331*D11E*pow(inE, 2)*out_of_plane + D33S)/(dx*dz);
vNW = (-0.33333333333333331*D11W*pow(inW, 2)*out_of_plane + D33N)/(dx*dz);
vNE = (0.33333333333333331*D11E*pow(inE, 2)*out_of_plane - D33N)/(dx*dz);
uSW = 0;
uSE = 0;
uNW = 0;
uNE = 0;
vSWW = 0;
vSEE = 0;
vNWW = 0;
vNEE = 0;
vSSW = 0;
vSSE = 0;
vNNW = 0;
vNNE = 0;
pW = -1/dx;
pE = 1.0/dx;
pSW = 0;
pSE = 0;
pNW = 0;
pNE = 0;


In [46]:
# Compressibility switch
comp = symbols('comp')
comp = 1

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])

# Stencil extension 1
vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE = symbols('vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE')
dofs_ext = Matrix([vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pS, pN, pSW, pSE, pNW, pNE = symbols('pS, pN, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pS, pN, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
DirE, DirW, NeuE, NeuW  = symbols('DirE, DirW, NeuE, NeuW')

# Divergences
divS      = out_of_plane*( (uSE-uSW)/dx + (vC -vS )/dz )   
divN      = out_of_plane*( (uNE-uNW)/dx + (vN -vC )/dz )  
divSW     = out_of_plane*( (uSW-uSWW)/dx + (vW -vSW )/dz )
divSE     = out_of_plane*( (uSEE-uSE)/dx + (vE -vSE )/dz )
divNW     = out_of_plane*( (uNW-uNWW)/dx + (vNW -vW )/dz )
divNE     = out_of_plane*( (uNEE-uNE)/dx + (vNE -vE )/dz ) 

# Deviatoric normal strain
ExxS      = ((uSE-uSW)/dx - comp*1/3*divS)
EzzS      = ((vC -vS )/dz - comp*1/3*divS)
ExxN      = ((uNE-uNW)/dx - comp*1/3*divN)
EzzN      = ((vN -vC )/dz - comp*1/3*divN)

# Additional missing stress tensor components via interpolation
GxzSW     = ((uSW  - uSSW)/dz + (vS  - vSW)/dx)
GxzSE     = ((uSE  - uSSE)/dz + (vSE - vS )/dx)
GxzNW     = ((uNNW - uNW )/dz + (vN  - vNW)/dx)
GxzNE     = ((uNNE - uNE )/dz + (vNE - vN )/dx)

# Shear strain rate
dVzdxE_in  = (vE - vC )/dx
dVzdxW_in  = (vC - vW )/dx
dVzdxE_Dir = (-vC - vC )/dx
dVzdxW_Dir = (vC - -vC )/dx
dVzdxE_Neu = (vC - vC )/dx
dVzdxW_Neu = (vC - vC )/dx
GxzE       = (uNE-uSE)/dz + inE*dVzdxE_in + DirE*dVzdxE_Dir + NeuE*dVzdxE_Neu
GxzW       = (uNW-uSW)/dz + inW*dVzdxW_in + DirW*dVzdxW_Dir + NeuW*dVzdxW_Neu

# Additional missing stress tensor components for interpolation
ExxNE     = ((uNEE-uNE)/dx - comp*1/3*(divNE))
ExxNW     = ((uNW-uNWW)/dx - comp*1/3*(divNW))
ExxSE     = ((uSEE-uSE)/dx - comp*1/3*(divSE))
ExxSW     = ((uSW-uSWW)/dx - comp*1/3*(divSW))
EzzNE     = ((vNE -vE )/dz - comp*1/3*(divNE))
EzzNW     = ((vNW -vW )/dz - comp*1/3*(divNW))
EzzSE     = ((vE -vSE )/dz - comp*1/3*(divSE))
EzzSW     = ((vW -vSW )/dz - comp*1/3*(divSW))

# INV 0
GxzN      = 0.25*( inW*GxzW + inE*GxzE + inNWv*GxzNW + inNEv*GxzNE )
GxzS      = 0.25*( inW*GxzW + inE*GxzE + inSWv*GxzSW + inSEv*GxzSE ) 
ExxE      = 0.25*( inS*ExxS + inN*ExxN + inNEc*ExxNE + inSEc*ExxSE )
ExxW      = 0.25*( inS*ExxS + inN*ExxN + inNWc*ExxNW + inSWc*ExxSW )
EzzE      = 0.25*( inS*EzzS + inN*EzzN + inNEc*EzzNE + inSEc*EzzSE )
EzzW      = 0.25*( inS*EzzS + inN*EzzN + inNWc*EzzNW + inSWc*EzzSW )
pE        = 0.25*( inS*pS   + inN*pN   + inNEc*pNE   + inSEc*pSE )
pW        = 0.25*( inS*pS   + inN*pN   + inNWc*pNW   + inSWc*pSW )

#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TzzS = TS[1].subs({D21:'D21S', D22:'D22S', D23:'D23S', D24:'D24S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TzzN = TN[1].subs({D21:'D21N', D22:'D22N', D23:'D23N', D24:'D24N'})
#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxzW = TW[2].subs({D31:'D31W', D32:'D32W', D33:'D33W', D34:'D34W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxzE = TE[2].subs({D31:'D31E', D32:'D32E', D33:'D33E', D34:'D34E'})
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW) - (pN - pS)/dz 
Fz = -Fz
for i in range(len(dofs)):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cVc)) + ';') 

vW = -D33W*inW/pow(dx, 2);
vC = (-pow(dx, 2)*(D22N + D22S)*(0.33333333333333331*out_of_plane - 1) + pow(dz, 2)*(D33E*(2*DirE + inE) + D33W*(2*DirW + inW)))/(pow(dx, 2)*pow(dz, 2));
vE = -D33E*inE/pow(dx, 2);
vS = D22S*(0.33333333333333331*out_of_plane - 1)/pow(dz, 2);
vN = D22N*(0.33333333333333331*out_of_plane - 1)/pow(dz, 2);
uSW = (0.33333333333333331*D22S*out_of_plane - D33W)/(dx*dz);
uSE = (-0.33333333333333331*D22S*out_of_plane + D33E)/(dx*dz);
uNW = (-0.33333333333333331*D22N*out_of_plane + D33W)/(dx*dz);
uNE = (0.33333333333333331*D22N*out_of_plane - D33E)/(dx*dz);
vSW = 0;
vSE = 0;
vNW = 0;
vNE = 0;
uSWW = 0;
uSEE = 0;
uNWW = 0;
uNEE = 0;
uSSW = 0;
uSSE = 0;
uNNW = 0;
uNNE = 0;
pS = -1/dz;
pN = 1.0/dz;
pSW = 0;
pSE = 0;
pNW = 0;
pNE = 0;
